In [111]:
!pip install gcsfs
from google.cloud import storage
from google.colab import drive

# Authenticate user and mount Google Drive
drive.mount('/content/drive')

# Set project and bucket names
project_id = 'cps585-finalproject-group7'
bucket_name = 'cps585-finalbucket'

# Create a storage client and get bucket object
storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(bucket_name)

# Set file path and name
file_path = '/content/drive/My Drive/project csv/imdb_movie_data1.csv'
file_name = 'imdb_movie_data1.csv'

# Create blob object and upload file to bucket
blob = bucket.blob(file_name)
blob.upload_from_filename(file_path)

print(f"{file_name} has been uploaded to {bucket_name} bucket in {project_id} project.")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
imdb_movie_data1.csv has been uploaded to cps585-finalbucket bucket in cps585-finalproject-group7 project.


This code installs the gcsfs library and imports the required modules for Google Cloud Storage and Google Drive.Then, it mounts the user's Google Drive and sets the project_id and bucket_name variables.

Next, it creates a Google Cloud Storage client using the storage.Client() method and retrieves the bucket object with the given bucket_name.After that, it sets the file_path and file_name variables, specifying the path and name of the file to be uploaded.

Finally, it creates a blob object representing the file in the bucket, uploads the file from the specified file path to the blob, and prints a success message.So, this code uploads the imdb_movie_data1.csv file from the user's Google Drive to the cps585-finalbucket bucket in the cps585-finalproject-group7 project on Google Cloud Storage.

In [112]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

dataset_id = "my_dataset"
dataset_ref = client.dataset(dataset_id)

client.delete_dataset(dataset_ref, delete_contents=True, not_found_ok=True)

dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"

dataset = client.create_dataset(dataset, timeout=30)


This code creates a new BigQuery dataset named "my_dataset" in the Google Cloud project specified by project_id variable. If the dataset already exists, it deletes it along with all its contents (delete_contents=True) and creates a new empty dataset with the same name. The dataset is created in the US region (dataset.location = "US") and the operation times out after 30 seconds (timeout=30). Finally, the dataset object is returned.

In [113]:
print(df.columns)

Index(['movie', 'year', 'certificate', 'genre.3', 'genre.3', 'genre.3',
       'imdb_rating', 'metascore', 'time_minute', 'vote', 'gross_earning'],
      dtype='object')


In [12]:
# Import necessary libraries
from google.cloud import bigquery
from google.cloud import storage
from google.cloud.exceptions import NotFound

# Set project_id, bucket_name and file_name variables
project_id = 'cps585-finalproject-group7'
bucket_name = 'cps585-finalbucket'
file_name = 'imdb_movie_data1.csv'
dataset_id = 'imdb_movies'
table_name = 'imdb_data'

# Initialize BigQuery client
bq_client = bigquery.Client(project=project_id)

This code imports the necessary libraries to work with Google Cloud's BigQuery and Storage services. Then, it sets the following variables:

project_id: the ID of the Google Cloud project.
bucket_name: the name of the Google Cloud Storage bucket where the data file is stored.
file_name: the name of the CSV file containing the data.
dataset_id: the name of the dataset that will be created in BigQuery to store the data.
table_name: the name of the table that will be created in the dataset to store the data.
Finally, it initializes a BigQuery client object bq_client with the given project ID.

In [15]:
# Initialize Storage client and get the bucket
storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(bucket_name)

# Check if file exists in bucket
try:
    blob = bucket.blob(file_name)
    blob.reload()
except NotFound:
    print('{} does not exist in {} bucket.'.format(file_name, bucket_name))
else:
    print('{} has been found in {} bucket.'.format(file_name, bucket_name))

imdb_movie_data1.csv has been found in cps585-finalbucket bucket.


This code is used to initialize the Google Cloud Storage client using the project ID provided in the project_id variable and get the specified bucket using the bucket_name variable.

After that, the code tries to check whether the file_name (which in this case is imdb_movie_data1.csv) exists in the bucket using a try-except block.

If the file exists, the code prints a message confirming that the file has been found in the specified bucket. If it does not exist, it prints a message saying that the file does not exist in the specified bucket.

This check is useful before proceeding with any operations related to the file in the bucket, such as loading it into a BigQuery table.

In [17]:
# Set dataset_ref
dataset_ref = bq_client.dataset(dataset_id)

# Create the dataset (if it doesn't exist)
try:
    bq_client.get_dataset(dataset_ref)
except NotFound:
    dataset = bigquery.Dataset(dataset_ref)
    dataset = bq_client.create_dataset(dataset)
    print('Created dataset {}.'.format(dataset_id))


This code sets a reference to the BigQuery dataset specified by the dataset_id variable. It then tries to get the dataset using the get_dataset method of the BigQuery client. If the dataset is not found, it creates a new dataset using the create_dataset method of the BigQuery client and prints a message indicating that the dataset has been created.

In [18]:
# Set table_id
table_id = '{}.{}.{}'.format(project_id, dataset_id, table_name)

# Set job_config for schema and delimiter
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField('movie', 'STRING'),
        bigquery.SchemaField('year', 'INTEGER'),
        bigquery.SchemaField('certificate', 'STRING'),
        bigquery.SchemaField('genre_1', 'STRING'),
        bigquery.SchemaField('genre_2', 'STRING'),
        bigquery.SchemaField('genre_3', 'STRING'),
        bigquery.SchemaField('imdb_rating', 'FLOAT'),
        bigquery.SchemaField('metascore', 'FLOAT'),
        bigquery.SchemaField('time_minute', 'STRING'),
        bigquery.SchemaField('vote', 'INTEGER'),
        bigquery.SchemaField('gross_earning', 'STRING'),
    ],
    skip_leading_rows=1,
    field_delimiter=','
)


This code sets the table_id variable which is a combination of the project_id, dataset_id and table_name. It also sets the job_config variable which includes the schema of the table to be created in BigQuery, as well as the delimiter and the number of rows to skip. The schema includes the field names and their corresponding data types, which is defined based on the columns of the CSV file that we are importing. The skip_leading_rows parameter specifies that the first row of the CSV file contains headers and should be skipped, while the field_delimiter parameter specifies that the fields are separated by commas.

In [24]:
# Define the path to the CSV file
file_path = '/content/drive/My Drive/project csv/imdb_movie_data1.csv'

# Replace "-" with "0" for metascore column in the CSV file
with open(file_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    rows = []
    for row in reader:
        row[7] = row[7].replace('-', '0')
        rows.append(row)

# Overwrite the CSV file with the modified data
with open(file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for row in rows:
        writer.writerow(row)

print('Done replacing "-" with "0" for metascore column in the CSV file')


Done replacing "-" with "0" for metascore column in the CSV file


This code replaces all occurrences of "-" with "0" in the metascore column of the CSV file located at the specified file path.

First, it opens the file using the open() function and reads its contents using the csv.reader() function. It then loops through each row in the file and replaces any "-" value in the 7th (or 8th) column (i.e., the metascore column) with "0". The modified rows are then appended to a new list called rows.

After modifying the data, the code then overwrites the original CSV file using the csv.writer() function to write each row to the file. Finally, it prints a message indicating that the replacement is done.